## 4.2. 손실 함수

In [2]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [3]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [9]:
def cross_entropy_error(y, t):
    return -np.sum(t * np.log(y + 1e-7))

cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [19]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_size = x_train.shape[0]
batch_size = 10

batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [20]:
def cross_entropy_error(y, t):
    if y.dim == 1:
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

## 4.3. 수치 미분

In [28]:
def numerical_diff(f, x):
    h = 1e-50
    return (f(x + h) - f(x)) / h

def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [4]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [9]:
## 편미분
def numerical_diff(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val
    
    return grad



In [10]:
numerical_diff(function_2, np.array([3.0, 4.0]))

ValueError: setting an array element with a sequence.

In [49]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for _ in range(step_num):
        grad = numerical_diff(f, x)
        x -= lr * grad
        
    return x

In [52]:
gradient_descent(function_2, np.array([-3.0, 4.0]), lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

### 4.4.2 신경망에서의 기울기

In [18]:
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
        
    def predict(self, x):
        return x @ self.W
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss
    
    
net = SimpleNet()
print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

np.argmax(p)

t = np.array([0, 0, 1])
net.loss(x, t)

[[ 2.11262919 -0.29311037  0.96155212]
 [ 0.01140821  0.79089731  1.08049269]]
[1.27784491 0.53594135 1.5493747 ]


0.7538580160377696

In [19]:
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)

[[ 2.11262919 -0.29311037  0.96155212]
 [ 0.01140821  0.79089731  1.08049269]]


ValueError: could not broadcast input array from shape (2,3) into shape (3,)